Importing required libraries and modules

In [ ]:
import pandas as pd
import numpy as np
import urllib
import os 
import requests
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
get_ipython().magic(u'matplotlib inline')
import time
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.PDB import *
import os, shutil
from biopandas.pdb import PandasPdb
from Bio import ExPASy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
import pymol as p


BLAST protocol - Collects target signatures from user and matches target record with PubChem data linked to target which is required to perform AutoQSAR based drug lead generation against the target 

In [25]:
# protein_query = "SMAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQ"

# nucleotide_query = "ATGGCGATGCTTCAGACGAATCTTGGCTTCATTACTTCTCCGACATTTCTGTGTCCGAAGCTTAAAGTCAAATTGAACTCTTATCTGTGGTTTAGCTATCGTTCTCAAGTTCAAAAACTGGATTTTTCGAAAAGGGTTAATAGAAGCTACAAAAGAGATGCTTTATTATTGTCAATCAAGTGTTCTTCATCGACTGGATTTGATAATAGCAATGTTGTTGTGAAGGAGAAGAGTGTATCTGTGATTCTTTTAGCTGGAGGTCAAGGCAAGAGAATGAAAATGAGTATGCCAAAGCAGTACATACCACTTCTTGGTCAGCCAATTGCTTTGTATAGCTTTTTCACGTTTTCACGTATGCCTGAAGTGAAGGAAATTGTAGTTGTATGTGATCCTTTTTTCAGAGACATTTTTGAAGAATACGAAGAATCAATTGATGTTGATCTTAGATTCGCTATTCCTGGCAAAGAAAGACAAGATTCTGTTTACAGTGGACTTCAGGAAATCGATGTGAACTCTGAGCTTGTTTGTATCCACGACTCTGCCCGACCATTGGTGAATACTGAAGATGTCGAGAAGGTCCTTAAAGATGGTTCCGCGGTTGGAGCAGCTGTACTTGGTGTTCCTGCTAAAGCTACAATCAAAGAGGTCAATTCTGATTCGCTTGTGGTGAAAACTCTCGACAGAAAAACCCTATGGGAAATGCAGACACCACAGGTGATCAAACCAGAGCTATTGAAAAAGGGTTTCGAGCTTGTAAAAAGTGAAGGTCTAGAGGTAACAGATGACGTTTCGATTGTTGAATACCTCAAGCATCCAGTTTATGTCTCTCAAGGATCTTATACAAACATCAAGGTTACAACACCTGATGATTTACTGCTTGCTGAGAGAATCTTGAGCGAGGACTCATGA"

query = Seq(input("Paste input in a form of FASTA sequence: "))

# def blast_input():
# my_query = Seq(nucleotide_query)
# query = SeqIO.read("sample.fasta", format="fasta")
try:
    print("Query sent to NCBIWWW against swissprot database")
    result_handle = NCBIWWW.qblast("blastp", "swissprot", query)
    blast_result = open("my_blast_result.xml", "w")
    blast_result.write(result_handle.read())
    blast_result.close()
    result_handle.close()
except ValueError:
    my_translated_query = my_query.translate()
    print(f"DNA/RNA sequence detected. Sequence translated to : {my_translated_query}")
    result_handle = NCBIWWW.qblast("blastp", "swissprot", my_translated_query)
    blast_result = open("my_blast_result.xml", "w")
    blast_result.write(result_handle.read())
    blast_result.close()
    result_handle.close()


# def read_blast_output():
result = open("my_blast_result.xml")
blast_records = NCBIXML.parse(result)
number_of_alignments = 0
hit_ids = []
for rec in blast_records:
    for alignment in rec.alignments:
        hit_ids.append(alignment.accession)
        number_of_alignments += 1
        if number_of_alignments == 3: # Take three top alignments and stop
            break
print(f"Top hits: {hit_ids}")
# return hit_ids
if len(hit_ids) == 0:
    print("No hits identified in the database. Check your FASTA input.")
# def read_target_db(input_list: list):
with open('target', 'r') as df:
    for line in df:
        line = line.split()
        try:
            for entry in hit_ids:
                if entry in line[2]:
                    print(f"Entry {entry} present in database!")
                    check_id = entry
                    break
        except:
            print("Target not found in database")

# if __name__ == '__main__':
#     blast_input()
#     read_target_db(read_blast_output())

Paste input in a form of FASTA sequence: MSTESMIRDVELAEEALPKKTGGPQGSRRCLFLSLFSFLIVAGATTLFCLLHFGVIGPQREEFPRDLSLISPLAQAVRSSSRTPSDKPVAHVVANPQAEGQLQWLNRRANALLANGVELRDNQLVVPSEGLYLIYSQVLFKGQGCPSTHVLLTHTISRIAVSYQTKVNLLSAIKSPCQRETPEGAEAKPWYEPIYLGGVFQLEKGDRLSAEINRPDYLDFAESGQVYFGIIAL
Query sent to NCBIWWW against swissprot database
Top hits: ['P01375', 'P33620', 'Q8HZD9']
Target not found in database
Target not found in database
Entry P01375 present in database!
Entry P01375 present in database!


In [26]:
link1 = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22P0C6U8%22}]},%22order%22:[%22activity,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22{PROTACXN_P0C6U8}_bioactivity_protein%22}"

In [37]:
link1 = link1.replace('P0C6U8', check_id)
print(link1) 


https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22P01375%22}]},%22order%22:[%22activity,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22{PROTACXN_P01375}_bioactivity_protein%22}


Data fetching from PubChem begins. Data linked to target of active compounds against target, their molecular descriptors and experimental activity value is fetched 

In [61]:
for i in range(2):
    try:
        req = requests.get(link1)
        url_content = req.content
        csv_file = open('downloaded1.csv', 'wb')
        csv_file.write(url_content)
        csv_file.close()
        print("Completed the Request")
        break
    #except IncompleteRead as I:
     #   print("Server Overloading , Proceeding")
      #  break
    except Exception as a:
        print(str(a)+" is the error , Trying {} time".format(i))
        continue
    else:
        break
else:
    print("something Wrong , Try running Again [refer error code for more]")

Completed the Request


In [62]:
data = pd.read_csv("downloaded1.csv",error_bad_lines=False)

data

,baid,activity,aid,sid,mid,cid,geneid,taxid,pmid,aidtype,...,protacxn,acname,acvalue,aidsrcname,aidname,cmpdname,targetname,targeturl,ecs,repacxn
0,87650010,Active,1852,24841471,0,164754,7124,9606,NaN,Other,...,NP_000585,NaN,NaN,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,Enniatin B,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
1,87650665,Active,1852,24818849,0,203396,7124,9606,NaN,Other,...,NP_000585,NaN,NaN,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,"4,5-Dibromo-2-phenylpyridazin-3(2H)-one",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
2,87650885,Active,1852,17510830,0,208719,7124,9606,NaN,Other,...,NP_000585,NaN,NaN,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,5-(p-Tolylazo)-8-quinolinol,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
3,87651570,Active,1852,24818561,0,224481,7124,9606,NaN,Other,...,NP_000585,NaN,NaN,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,"2-Chloro-N,N-diphenylacetamide",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
4,87655148,Active,1852,844338,0,646658,7124,9606,NaN,Other,...,NP_000585,NaN,NaN,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,"2-(7-Butyl-1,3-dimethyl-2,6-dioxo-2,3,6,7-tetr...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6607,460544356,Unspecified,1186217,312348743,0,874861,7124,9606,25078315.0,Literature-derived,...,P01375,NaN,NaN,ChEMBL,Inhibition of TNFalpha in HEK293T cells transf...,N-(3-Chloro-4-hydroxyphenyl)-4-fluorobenzenesu...,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
6608,460544358,Unspecified,1186217,312370226,0,1137620,7124,9606,25078315.0,Literature-derived,...,P01375,NaN,NaN,ChEMBL,Inhibition of TNFalpha in HEK293T cells transf...,"(3,5-Dichloro-4-hydroxyphenyl)(2-naphthylsulfo...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
6609,460865365,Unspecified,733377,103562257,0,5327044,7124,9606,24900576.0,Literature-derived,...,P01375,NaN,NaN,ChEMBL,Binding affinity to GST-tagged human TNF-alpha...,"6,7-Dimethyl-3-[(methyl{2-[methyl({1-[3-(trifl...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375
6610,460865369,Unspecified,733378,103562257,0,5327044,7124,9606,24900576.0,Literature-derived,...,P01375,NaN,NaN,ChEMBL,Ratio of Kd for GST-tagged human TNF-alpha in ...,"6,7-Dimethyl-3-[(methyl{2-[methyl({1-[3-(trifl...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375


In [63]:
#data cleaning up
new_data = data[['cid','acvalue']]
new_data = new_data.dropna()
LSTM_data = data['cid']

In [64]:
except_val=0
cid_value = new_data['cid'].to_list()
PIC50_value = (-np.log10(new_data['acvalue']*10**-6)).to_list()

PIC50_value = pd.DataFrame(PIC50_value,columns = ["y"])
y_data = PIC50_value

In [65]:
#WebAPI PubChem

link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/new_link_me/property/MolecularWeight,HeavyAtomCount,XLOGP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA/CSV"
link_fixed = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/new_link_me/property/MolecularWeight,HeavyAtomCount,XLOGP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA/CSV"
sub_link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/cid/replaceme/cids/TXT"
sub_link_fixed = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/cid/replaceme/cids/TXT"
#the link with the required properties.
counter = 1

In [66]:
final_data_frame = pd.DataFrame()
x_data = pd.DataFrame()
cid_main_counter=0 
for x in cid_value:
    for iter_x in range(1000):
        try:    
            link = link_fixed
            link = link.replace("new_link_me",str(x))
            data1 = pd.read_csv(link)
            data1 = pd.DataFrame(data1)
            x_data = x_data.append(data1)
            data1 = 0
        except Exception as e:
            print("Exception Encountered as {} .! , Trying again !Iteration : {}".format(str(e),str(iter_x)))
            time.sleep(5)
            continue
        else:
            break
    
    else:
        print("Something Wrong with the Trials ! Restart The Algorithm!")

In [67]:
x_data_saved = x_data
x_data = x_data_saved
n_cid=[]

In [68]:
for i in cid_value:
    max_tries = 10
    
    for iter_ in range(max_tries):
        try:
            #print("inside Try")
            #time.sleep(5)


            link = link_fixed
            link = link.replace("new_link_me",str(i))
            f_data = pd.read_csv(link) 
            
            sub_link = sub_link_fixed
            sub_link = sub_link.replace("replaceme",str(i))
            res = urllib.request.urlopen(sub_link)
            data_sub = res.read()
            data_sub = str(data_sub)
            data_sub = data_sub.replace("\\n",",") ; data_sub = data_sub.replace('b'," ")
            data_sub = data_sub.replace("'",""); data_sub = data_sub.replace(" ","")
            n_count_loop = 0 
            #time.sleep(3)
            #print("Going for the loop")
            for j in data_sub.split(","):
                n_count_loop+=1
                if(len(j)>1):
                    if(n_count_loop<=30):
                        n_cid.append(j)
                    else:
                        break
                else:
                    pass
             
        except:
            if(except_val>=15):
                break
            else:
                print("Re-Trying")
                except_val+=1
                #time.sleep(3)
                continue
        else:
            break
    '''else:
        print("Either The Network is Down(So no point in continuing ) , Or Some Uknown Error Spotted!. Refer Error Code , Continuing with Fetched Data")
        time.sleep(5)
        break'''
    
    
    '''final_data_frame = final_data_frame.append(f_data)
    cid_main_counter+=1
    final_data_frame.to_csv("fdf1.csv")
         
   

    print("Main cid no: {}".format(cid_main_counter))
    #time.sleep(5)'''
    



Re-Trying
Re-Trying


In [69]:
k_count = 0
t_count = 0
phase_count=0
f_data = pd.DataFrame()

In [ ]:
#data mining PubChem through web API

for k in n_cid:
    k_count+=1
    t_count+=1
    phase_count+=1
    for sub_iter in range(10):
            try:
                if(t_count>=50):
                    #print("taking Rest")
                    time.sleep(15)
                    t_count=0
                    
                #print("Inside Try"+str(k_count))
            
                link = link_fixed
                link = link.replace("new_link_me",str(k))
                #print("replacement done"+str(k_count))
                f_data_df = pd.read_csv(link)
                f_data = f_data.append(f_data_df)
                print("Passed Without Exception"+str(k_count))
                break

            except Exception as e:
                print(str(e) + "Encountered , Please Wait :+ "+str(k_count))
                time.sleep(20)
                continue

    final_data_frame = final_data_frame.append(f_data)
    cid_main_counter+=1
    final_data_frame.to_csv("fdf1.csv")
    print("Data Fetching Continued"+ str(k_count))
    
    


In [71]:
final_data_frame.drop_duplicates(inplace=True)
print(final_data_frame.shape)
print("The Final Training DataSet: X_data = {} , Y_data = {}".format(x_data.shape,y_data.shape))
print("The Final Data For Preditcion is : {}".format(final_data_frame.shape))

(3861, 9)
The Final Training DataSet: X_data = (2461, 9) , Y_data = (2461, 1)
The Final Data For Preditcion is : (3861, 9)


In [73]:
n_cid.extend(LSTM_data)
print(len(n_cid))

81879


In [74]:
try:
    final_data_frame.reset_index(inplace=True)
    final_data_frame.drop('index',axis=1,inplace=True)
except:
    pass


In [75]:
final_data_frame_reset = final_data_frame
x_data_reset = x_data
y_data_reset = y_data

Auto QSAR algorithm

In [76]:
'''#RUN TO RESET ONLY : TESTING CAUTION
final_data_frame = final_data_frame_reset
x_data = x_data_reset
y_data = y_data_reset'''

'#RUN TO RESET ONLY : TESTING CAUTION\nfinal_data_frame = final_data_frame_reset\nx_data = x_data_reset\ny_data = y_data_reset'

In [77]:
#data preparation for QSAR model building
x_data = pd.DataFrame(x_data)

In [78]:
y_data = pd.DataFrame(y_data)

In [79]:
print("X :- "+str(x_data.shape)+" Y :- "+str(y_data.shape) )

X :- (2461, 9) Y :- (2461, 1)


In [80]:
train_file = pd.DataFrame()
x_data = x_data.astype("float64")
y_data = y_data.astype("float64")

In [81]:
#molecular features for AutoQSAR model building
x_data.head()

,CID,MolecularWeight,HeavyAtomCount,XLogP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA
0,649032.0,437.5,29.0,2.1,640.0,8.0,437.130380,8.0,167.0
0,653676.0,448.6,30.0,2.6,697.0,7.0,448.135131,6.0,154.0
0,666267.0,445.0,31.0,2.8,658.0,5.0,444.204052,6.0,64.9
0,3237441.0,449.4,30.0,3.8,685.0,11.0,449.046868,5.0,105.0
0,3123424.0,372.5,26.0,3.7,498.0,4.0,372.161997,8.0,92.5


In [82]:
#experimental inhibition value for model building
y_data.head()

,y
0,4.886057
1,5.673664
2,5.474955
3,4.987163
4,4.752027


In [83]:
x_data.head()

,CID,MolecularWeight,HeavyAtomCount,XLogP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA
0,649032.0,437.5,29.0,2.1,640.0,8.0,437.130380,8.0,167.0
0,653676.0,448.6,30.0,2.6,697.0,7.0,448.135131,6.0,154.0
0,666267.0,445.0,31.0,2.8,658.0,5.0,444.204052,6.0,64.9
0,3237441.0,449.4,30.0,3.8,685.0,11.0,449.046868,5.0,105.0
0,3123424.0,372.5,26.0,3.7,498.0,4.0,372.161997,8.0,92.5


In [84]:
new = y_data['y'].to_list()

In [85]:
train_file  = x_data
train_file['y'] = new

In [86]:
cid_reg_list=train_file['CID'].to_list()
train_file.drop('CID',axis=1,inplace =True)


In [87]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')
train_file

#data set of molecular features and experimental inhibition value for QSAR model building

,MolecularWeight,HeavyAtomCount,XLogP,Complexity,HBondAcceptorCount,MonoisotopicMass,RotatableBondCount,TPSA,y
0,437.50,29.0,2.1,640.0,8.0,437.130380,8.0,167.0,4.886057
0,448.60,30.0,2.6,697.0,7.0,448.135131,6.0,154.0,5.673664
0,445.00,31.0,2.8,658.0,5.0,444.204052,6.0,64.9,5.474955
0,449.40,30.0,3.8,685.0,11.0,449.046868,5.0,105.0,4.987163
0,372.50,26.0,3.7,498.0,4.0,372.161997,8.0,92.5,4.752027
...,...,...,...,...,...,...,...,...,...
0,390.40,27.0,0.7,609.0,8.0,390.127423,4.0,92.1,4.522879
0,153.14,11.0,1.3,160.0,4.0,153.042593,1.0,83.6,1.686133
0,301.72,19.0,2.8,392.0,5.0,300.997570,3.0,74.8,4.000000
0,368.20,23.0,4.6,498.0,4.0,366.983670,3.0,74.8,4.000000


In [88]:
x = list(train_file.columns)
x = x[:-1]

x_ = train_file[x].values
y_ = train_file['y'].values

In [89]:
train_file.describe()
x_ = x_.astype("float64")
y_ = y_.astype("float64")

In [90]:
X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.1, random_state=0)


In [91]:
#AutoQSAR workflow

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [92]:
scaler = preprocessing.StandardScaler()
degree=1
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=1, include_bias=True,
                                    interaction_only=False, order='C')),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [93]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
main_r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
max_ = main_r2 
max_

0.478917275165057

In [94]:
from itertools import combinations
comb_list = [[]]

In [95]:
#creating combinations , function
def sub(arr,r):
    global comb_list
    for i in r:
        comb = list(combinations(arr,i))
        comb_list.append(comb)
    return comb_list
newone = 0
newone = sub(x , [2,3,4,5,6])
del newone[0]

In [96]:
#QSAR model building based on linear multivariate regression for all possible combinations of descriptors and choosing the model with best statistical quality 

In [97]:
#degree 1 
coef_dict  = 0
loop_index = 0
coef_dict = [[]]
r2_score_new =[]
max_r2 = []
index_r2 =[]
for i in range(0,len(newone)):
    
    index_r=0
    for combi_ in newone[loop_index]:
        
        #print(combi_)
        features = list(combi_)
        features_=train_file[features].values
        output_=train_file['y'].values
        X_train, X_test, y_train, y_test = train_test_split(features_, output_, test_size=0.1, random_state=0)
        scaler = preprocessing.StandardScaler()
        degree=1
        polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
        polyreg_scaled.fit(X_train,y_train)
        y_pred = polyreg_scaled.predict(X_test)
        r2_score_  = r2_score(y_test, y_pred, multioutput='uniform_average')
        r2_score_new.append(r2_score_)
    loop_index+=1
        
    
    max_r2.append(max(r2_score_new))
    index_r = r2_score_new.index(max(r2_score_new))
    index_r2.append(index_r)
    r2_score_new =[]

In [98]:
#Extracting the best value so far using the regressor
sec_index = max_r2.index(max(max_r2))
fir_index = index_r2[sec_index]


In [99]:
if main_r2 > max(max_r2):
    r2_features = x
else:
    
    features  = newone[sec_index][fir_index]
    maxi_r2 = max(max_r2)

In [100]:
print("R2 - degree1 = " + str(maxi_r2)+str("\n")+str("Features = ")+str(features))
reg_max_r2 = max(max_r2)

R2 - degree1 = 0.4792535624044717
Features = ('HeavyAtomCount', 'XLogP', 'Complexity', 'MonoisotopicMass', 'RotatableBondCount', 'TPSA')


In [101]:
#We have the best degree 1 model
model_dict = {}
model_dict[1]=[maxi_r2,features]

In [102]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')

In [103]:
#QSAR model building based on non-linear multivariate regression(degree 2) for all possible combinations of descriptors and choosing the model with best statistical quality 
coef_dict  = 0
loop_index = 0
coef_dict = [[]]
r2_score_new =[]
max_r2 = []
index_r2 =[]
for i in range(0,len(newone)):
    
    index_r=0
    for combi_ in newone[loop_index]:
        
        #print(combi_)
        features = list(combi_)
        features_=train_file[features].values
        output_=train_file['y'].values
        X_train, X_test, y_train, y_test = train_test_split(features_, output_, test_size=0.1, random_state=0)
        scaler = preprocessing.StandardScaler()
        degree=2
        polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
        polyreg_scaled.fit(X_train,y_train)
        y_pred = polyreg_scaled.predict(X_test)
        r2_score_  = r2_score(y_test, y_pred, multioutput='uniform_average')
        r2_score_new.append(r2_score_)
    loop_index+=1
        
    
    max_r2.append(max(r2_score_new))
    index_r = r2_score_new.index(max(r2_score_new))
    index_r2.append(index_r)
    r2_score_new =[]

In [104]:
#Extracting the best value so far using the regressor
sec_index = max_r2.index(max(max_r2))
fir_index = index_r2[sec_index]

In [105]:
if main_r2 > max(max_r2):
    r2_features = x
else:
    
    features  = newone[sec_index][fir_index]
    maxi_r2 = max(max_r2)

In [106]:
print("R2 - degree2 = " + str(maxi_r2)+str("\n")+str("Features = ")+str(features))
reg_max_r2 = max(max_r2)

R2 - degree2 = 0.5811898180540496
Features = ('HeavyAtomCount', 'XLogP', 'Complexity', 'MonoisotopicMass', 'RotatableBondCount', 'TPSA')


In [107]:
model_dict[2]=[maxi_r2,features]

In [108]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')

In [109]:
model_dict

{1: [0.4792535624044717,
  ('HeavyAtomCount',
   'XLogP',
   'Complexity',
   'MonoisotopicMass',
   'RotatableBondCount',
   'TPSA')],
 2: [0.5811898180540496,
  ('HeavyAtomCount',
   'XLogP',
   'Complexity',
   'MonoisotopicMass',
   'RotatableBondCount',
   'TPSA')]}

In [110]:
#QSAR model building based on non-linear multivariate regression(degree 3) for all possible combinations of descriptors and choosing the model with best statistical quality 


import pandas as pd

import numpy as np

import urllib

import os 

import requests

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

import seaborn as seabornInstance 

from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LinearRegression

from sklearn import metrics

get_ipython().magic(u'matplotlib inline')

import time

link1 = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22where%22:{%22ands%22:[{%22protacxn%22:%22notnull%22},{%22cid%22:%22notnull%22},{%22repacxn%22:%22P0C6U8%22}]},%22order%22:[%22activity,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PROTACXN_P0C6U8_bioactivity_protein%22}"

for i in range(0,2):

    try:

        os.remove('downloaded1.csv')

        #print("Deleted old File")

        break

    except Exception as e:

        #print("No file ")

        break

    else:

        break

for i in range(2):

    try:

        data = pd.read_csv(link1)

        break

    #except IncompleteRead as I:

     #   print("Server Overloading , Proceeding")

      #  break

    except Exception as a:

        print(str(a)+" is the error , Trying {} time".format(i))

        continue

    else:

        break

else:

    print("something Wrong , Try running Again [refer error code for more]")

#data = pd.read_csv("downloaded1.csv",error_bad_lines=False)

data = pd.DataFrame(data)

coef_dict  = 0
loop_index = 0
coef_dict = [[]]
r2_score_new =[]
max_r2 = []
index_r2 =[]
for i in range(0,len(newone)):
    
    index_r=0
    for combi_ in newone[loop_index]:
        
        #print(combi_)
        features = list(combi_)
        features_=train_file[features].values
        output_=train_file['y'].values
        X_train, X_test, y_train, y_test = train_test_split(features_, output_, test_size=0.1, random_state=0)
        scaler = preprocessing.StandardScaler()
        degree=3
        polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
        polyreg_scaled.fit(X_train,y_train)
        y_pred = polyreg_scaled.predict(X_test)
        r2_score_  = r2_score(y_test, y_pred, multioutput='uniform_average')
        r2_score_new.append(r2_score_)
    loop_index+=1
        
    
    max_r2.append(max(r2_score_new))
    index_r = r2_score_new.index(max(r2_score_new))
    index_r2.append(index_r)
    r2_score_new =[]

In [111]:
#Extracting the best value so far using the regressor
sec_index = max_r2.index(max(max_r2))
fir_index = index_r2[sec_index]

In [112]:
if main_r2 > max(max_r2):
    r2_features = x
else:
    
    features  = newone[sec_index][fir_index]
    maxi_r2 = max(max_r2)

In [113]:
print("R2 - degree3 = " + str(maxi_r2)+str("\n")+str("Features = ")+str(features))
reg_max_r2 = max(max_r2)

R2 - degree3 = 0.5915122224245102
Features = ('HeavyAtomCount', 'Complexity', 'HBondAcceptorCount', 'MonoisotopicMass', 'RotatableBondCount', 'TPSA')


In [114]:
model_dict[3]=[maxi_r2,features]

In [115]:
for x in train_file.isnull().any():
    if x == True:
        train_file = train_file.fillna(method='ffill')

In [116]:
model_dict

{1: [0.4792535624044717,
  ('HeavyAtomCount',
   'XLogP',
   'Complexity',
   'MonoisotopicMass',
   'RotatableBondCount',
   'TPSA')],
 2: [0.5811898180540496,
  ('HeavyAtomCount',
   'XLogP',
   'Complexity',
   'MonoisotopicMass',
   'RotatableBondCount',
   'TPSA')],
 3: [0.5915122224245102,
  ('HeavyAtomCount',
   'Complexity',
   'HBondAcceptorCount',
   'MonoisotopicMass',
   'RotatableBondCount',
   'TPSA')]}

In [117]:
top_cid_mix_df = pd.DataFrame()

In [118]:
#Prediction with the validated model
model_dict[1]

[0.4792535624044717,
 ('HeavyAtomCount',
  'XLogP',
  'Complexity',
  'MonoisotopicMass',
  'RotatableBondCount',
  'TPSA')]

In [119]:
features = list(model_dict[1][1])
features

['HeavyAtomCount',
 'XLogP',
 'Complexity',
 'MonoisotopicMass',
 'RotatableBondCount',
 'TPSA']

In [120]:
x_trained =train_file[features].values
y_trained = train_file['y'].values
x_trained.shape,y_trained.shape

((2461, 6), (2461,))

In [121]:
X_train, X_test, y_train, y_test = train_test_split(x_trained, y_trained, test_size=0.1, random_state=0)

In [122]:
#poly regression - degree 1
scaler = preprocessing.StandardScaler()
degree=1
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=1, include_bias=True,
                                    interaction_only=False, order='C')),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [123]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
r2_score(y_test, y_pred, multioutput='uniform_average')

0.4792535624044717

In [124]:
final_data_frameF1 = final_data_frame[final_data_frame['MolecularWeight'] <= 500]
final_data_frameF2 = final_data_frameF1[final_data_frameF1['XLogP'] <=5.0]

In [125]:
pred_data_cid = final_data_frameF2["CID"].to_list()

In [126]:
#features.append('CID')
new_features = features.append("CID")
features

['HeavyAtomCount',
 'XLogP',
 'Complexity',
 'MonoisotopicMass',
 'RotatableBondCount',
 'TPSA',
 'CID']

In [127]:
final_data_frame1 = final_data_frameF2[features]
for x in final_data_frame1.isnull().any():
    if x == True:
        final_data_frame1 = final_data_frame1.fillna(method='ffill')
final_data_frame1.drop(columns='CID',inplace=True)
final_data_frame1.shape

/home/ben/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(3254, 6)

In [128]:
final_pred_1 = polyreg_scaled.predict(final_data_frame1)

In [129]:
final_pred_1 = list(final_pred_1)
final_data_frame1['CID'] = pred_data_cid
final_data_frame1['y_'] = final_pred_1

/home/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [130]:
saved_final_data_frame1 = final_data_frame1
sorted_final_df = final_data_frame1.sort_values('y_',ascending =0)
final_larg= sorted_final_df.head(50)

In [131]:
final_larg = final_larg[['CID','y_']]
top_cid = final_larg["CID"].to_list()

In [132]:
print("The Top 50 Drug Leads Which are identified with PubChem cid's are (USING Linear REGRESSION): ")
itter_count = 0
for itter in top_cid:
    itter_count+=1
    
    print(str(itter_count)+" : "+str(itter))

The Top 50 Drug Leads Which are identified with PubChem cid's are (USING Linear REGRESSION): 
1 : 364723
2 : 356283
3 : 3074872
4 : 70681601
5 : 129780132
6 : 3157191
7 : 711253
8 : 387846
9 : 356284
10 : 613997
11 : 76308151
12 : 1312315
13 : 150946647
14 : 72696275
15 : 70690018
16 : 132466207
17 : 364721
18 : 1397732
19 : 708445
20 : 150361908
21 : 23248475
22 : 702048
23 : 129882848
24 : 752161
25 : 70690020
26 : 40206146
27 : 702080
28 : 752162
29 : 2114010
30 : 153908250
31 : 11454798
32 : 654371
33 : 70690019
34 : 70694238
35 : 647632
36 : 136376421
37 : 6914643
38 : 118736396
39 : 70687997
40 : 654035
41 : 3213399
42 : 44593120
43 : 57894519
44 : 70687994
45 : 150613368
46 : 14640247
47 : 141074326
48 : 118986721
49 : 102059679
50 : 337362


In [133]:
print("r2 = "+str(r2_score(y_test, y_pred, multioutput='uniform_average')))

r2 = 0.4792535624044717


In [134]:
top_cid_mix_df['Degree 1'] = top_cid

In [135]:
final_cid_1 = pd.DataFrame()
final_cid_1['CID'] = top_cid
final_cid_1.to_csv("final_cid_degree_1.csv")

In [136]:
model_dict[2]

[0.5811898180540496,
 ('HeavyAtomCount',
  'XLogP',
  'Complexity',
  'MonoisotopicMass',
  'RotatableBondCount',
  'TPSA')]

In [137]:
features = list(model_dict[2][1])
features

['HeavyAtomCount',
 'XLogP',
 'Complexity',
 'MonoisotopicMass',
 'RotatableBondCount',
 'TPSA']

In [138]:
x_trained =train_file[features].values
y_trained = train_file['y'].values
x_trained.shape,y_trained.shape

((2461, 6), (2461,))

In [139]:
X_train, X_test, y_train, y_test = train_test_split(x_trained, y_trained, test_size=0.1, random_state=0)

In [140]:
#poly regression - degree 2
scaler = preprocessing.StandardScaler()
degree=2
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [141]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
r2_score(y_test, y_pred, multioutput='uniform_average')

0.5811898180540496

In [142]:
final_data_frameF1 = final_data_frame[final_data_frame['MolecularWeight'] <= 500]
final_data_frameF2 = final_data_frameF1[final_data_frameF1['XLogP'] <=5.6]

In [143]:
pred_data_cid = final_data_frameF2["CID"].to_list()

In [144]:
#features.append('CID')
new_features = features.append("CID")
features

['HeavyAtomCount',
 'XLogP',
 'Complexity',
 'MonoisotopicMass',
 'RotatableBondCount',
 'TPSA',
 'CID']

In [145]:
final_data_frame1 = final_data_frameF2[features]
for x in final_data_frame1.isnull().any():
    if x == True:
        final_data_frame1 = final_data_frame1.fillna(method='ffill')
final_data_frame1.drop(columns='CID',inplace=True)
final_data_frame1.shape

/home/ben/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(3485, 6)

In [146]:
final_pred_1 = polyreg_scaled.predict(final_data_frame1)
final_pred_1 = list(final_pred_1)
final_data_frame1['CID'] = pred_data_cid
final_data_frame1['y_'] = final_pred_1

/home/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [147]:
saved_final_data_frame1 = final_data_frame1
sorted_final_df = final_data_frame1.sort_values('y_',ascending =0)
final_larg= sorted_final_df.head(50)

In [148]:
final_larg = final_larg[['CID','y_']]
top_cid = final_larg["CID"].to_list()

In [149]:
print("The Top 50 Drug Leads Which are identified with PubChem cid's are (USING DEGREE 2 REGRESSION): ")
itter_count = 0
for itter in top_cid:
    itter_count+=1
    
    print(str(itter_count)+" : "+str(itter))

The Top 50 Drug Leads Which are identified with PubChem cid's are (USING DEGREE 2 REGRESSION): 
1 : 3074872
2 : 70681601
3 : 76308151
4 : 144048489
5 : 11454798
6 : 118736396
7 : 70690018
8 : 44593120
9 : 6914643
10 : 654035
11 : 654371
12 : 70690020
13 : 3213399
14 : 133314
15 : 70690019
16 : 70694238
17 : 126459002
18 : 70687997
19 : 136376421
20 : 70687994
21 : 53802
22 : 150613368
23 : 72696275
24 : 40206146
25 : 132466207
26 : 1312315
27 : 364721
28 : 44452251
29 : 11027418
30 : 71811258
31 : 142723051
32 : 3157191
33 : 134146806
34 : 150432640
35 : 44452332
36 : 24980701
37 : 1454593
38 : 648426
39 : 24978980
40 : 11547797
41 : 647632
42 : 24749927
43 : 134150252
44 : 131979407
45 : 1896005
46 : 134154648
47 : 134139036
48 : 24750165
49 : 6485862
50 : 145974899


In [150]:
top_cid_mix_df['Degree 2'] = top_cid

In [151]:
print("r2 = "+str(r2_score(y_test, y_pred, multioutput='uniform_average')))
final_cid_2 = pd.DataFrame()
final_cid_2['CID'] = top_cid
final_cid_2.to_csv("final_cid_degree_2.csv")

r2 = 0.5811898180540496


In [152]:
model_dict[3]

[0.5915122224245102,
 ('HeavyAtomCount',
  'Complexity',
  'HBondAcceptorCount',
  'MonoisotopicMass',
  'RotatableBondCount',
  'TPSA')]

In [153]:
features = list(model_dict[3][1])
features

['HeavyAtomCount',
 'Complexity',
 'HBondAcceptorCount',
 'MonoisotopicMass',
 'RotatableBondCount',
 'TPSA']

In [154]:
x_trained =train_file[features].values
y_trained = train_file['y'].values
x_trained.shape,y_trained.shape

((2461, 6), (2461,))

In [155]:
X_train, X_test, y_train, y_test = train_test_split(x_trained, y_trained, test_size=0.1, random_state=0)

In [156]:
#poly regression - degree 3
scaler = preprocessing.StandardScaler()
degree=3
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),scaler,LinearRegression())
polyreg_scaled.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [157]:
#R2 Score
y_pred = polyreg_scaled.predict(X_test)
r2_score(y_test, y_pred, multioutput='uniform_average')

0.5915122224245102

In [158]:
final_data_frameF1 = final_data_frame[final_data_frame['MolecularWeight'] <= 500]
final_data_frameF2 = final_data_frameF1[final_data_frameF1['XLogP'] <=5.6]

In [159]:
pred_data_cid = final_data_frameF2["CID"].to_list()

In [160]:
#features.append('CID')
new_features = features.append("CID")
features

['HeavyAtomCount',
 'Complexity',
 'HBondAcceptorCount',
 'MonoisotopicMass',
 'RotatableBondCount',
 'TPSA',
 'CID']

In [161]:
final_data_frame1 = final_data_frameF2[features]
for x in final_data_frame1.isnull().any():
    if x == True:
        final_data_frame1 = final_data_frame1.fillna(method='ffill')
final_data_frame1.drop(columns='CID',inplace=True)
final_data_frame1.shape

/home/ben/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(3485, 6)

In [162]:
final_pred_1 = polyreg_scaled.predict(final_data_frame1)
final_pred_1 = list(final_pred_1)
final_data_frame1['CID'] = pred_data_cid
final_data_frame1['y_'] = final_pred_1

/home/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [163]:
saved_final_data_frame1 = final_data_frame1
sorted_final_df = final_data_frame1.sort_values('y_',ascending =0)
final_larg= sorted_final_df.head(50)

In [164]:
final_larg = final_larg[['CID','y_']]
top_cid = final_larg["CID"].to_list()

In [165]:
print("The Top 50 Drug Leads Which are identified with PubChem cid's are (USING DEGREE 3 REGRESSION): ")
itter_count = 0
for itter in top_cid:
    itter_count+=1
    
    print(str(itter_count)+" : "+str(itter))

The Top 50 Drug Leads Which are identified with PubChem cid's are (USING DEGREE 3 REGRESSION): 
1 : 148332828
2 : 24830655
3 : 3473521
4 : 150946647
5 : 2134718
6 : 44593120
7 : 752162
8 : 153908250
9 : 2134378
10 : 55080117
11 : 752161
12 : 132466207
13 : 131983136
14 : 3157191
15 : 1312315
16 : 387846
17 : 711253
18 : 56927101
19 : 11547797
20 : 129780132
21 : 144415584
22 : 18591132
23 : 703093
24 : 52102698
25 : 708445
26 : 249748
27 : 73852568
28 : 144048489
29 : 364723
30 : 133314
31 : 2133311
32 : 1397732
33 : 76308151
34 : 356283
35 : 2133307
36 : 3074872
37 : 4487604
38 : 364721
39 : 702048
40 : 2833930
41 : 1899741
42 : 7120706
43 : 5761940
44 : 3116128
45 : 356284
46 : 22588269
47 : 72651363
48 : 60111994
49 : 147042332
50 : 70681601


In [166]:
top_cid_mix_df['Degree 3'] = top_cid

In [167]:
print("r2 = "+str(r2_score(y_test, y_pred, multioutput='uniform_average')))
final_cid_2 = pd.DataFrame()
final_cid_2['CID'] = top_cid
final_cid_2.to_csv("final_cid_degree_2.csv")

r2 = 0.5915122224245102


 THE END - COMPOUND FETCHING

In [168]:
top_cid_mix_df

,Degree 1,Degree 2,Degree 3
0,364723,3074872,148332828
1,356283,70681601,24830655
2,3074872,76308151,3473521
3,70681601,144048489,150946647
4,129780132,11454798,2134718
5,3157191,118736396,44593120
6,711253,70690018,752162
7,387846,44593120,153908250
8,356284,6914643,2134378
9,613997,654035,55080117


In [ ]:
top_cid_mix_df.to_csv("TOP_CID_123.csv")

top_cid = top_cid_mix_df['Degree 3'].tolist()

Automated In Silico modelling for drug leads generated from PubChem

In [32]:
class ResSelect(Select):
    def accept_residue(self, residue):
        if is_aa(residue):
            return 1
        else:
            return 0


class ProteinPreparer:



    def __init__(self, protein: str, dir: str):
        self.protein = protein
        self.tmpdir = dir
        self.filename = self.tmpdir + '/pdb{}.ent'.format(self.protein)

    def prepare_protein(self):
        parser = PDBParser()
        io = PDBIO()
        PDBList().retrieve_pdb_file(pdb_code=self.protein, pdir=self.tmpdir,
                                    file_format='pdb')  # saves pdb in a form of ent file
        ipdb = parser.get_structure('ipdb', self.filename)  # Input pdb as a self.filename
        io.set_structure(ipdb)  # Setting structure for input pdb
        io.save(self.tmpdir + '/'+self.protein + '.pdb', ResSelect(),
                preserve_atom_numbering=True)  # saves the cleaned pdb
        os.system(prepare_protein_path + self.tmpdir + '/{}.pdb'.format(
            self.protein))  # prepares the structure by adding polar hydrogens and adding gesteiger charges
        shutil.move(self.protein + '.pdbqt', self.tmpdir)

prepare_ligand_path = '~/MGLTools-1.5.6/bin/pythonsh ~/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py -A bonds_hydrogens -U nphs_lps -l'

class LigandPreparer:

    def __init__(self, ligand_file: str, dir: str):
        self.ligand = ligand_file
        self.tmpdir = dir

    def prepare_ligand(self):
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/SDF'.format(self.ligand)
        urllib.request.urlretrieve(url, self.tmpdir + '/{}'.format(self.ligand + '.sdf')) # downloads the file
        os.system('obabel {} -O {} --gen3d'.format(
            self.tmpdir + '/' + self.ligand + '.sdf', self.tmpdir + '/' + self.ligand + 'prep.pdb')
        ) # generates 3d coords
        os.system(
            'obminimize -ff GAFF {} > {}'.format(
                self.tmpdir + '/' + self.ligand + 'prep.pdb', self.tmpdir + '/' + self.ligand + '.pdb'
                                                 )
        ) # minimizes using GAFF
        os.system(prepare_ligand_path + '{}'.format(self.tmpdir + '/' + self.ligand + '.pdb'
                                                    )) # adds charges, sets rotatable bonds
        shutil.move(self.ligand + '.pdbqt', self.tmpdir)

class VinaDocker:

    def __init__(self, ligentry: str, protentry: str, protein_pdbqt: str, ligand_pdbqt: str, dir: str):
        self.protein = protein_pdbqt + '.pdbqt'
        self.protpdb = protentry + '.pdb'
        self.protname = os.path.basename(self.protein)
        self.ligand = ligand_pdbqt + '.pdbqt'
        self.ligname = os.path.basename(self.ligand)
        self.tmpdir = dir
        self.docklog = './results/' + protentry + '_' + ligentry + '_docking.log' # !!
        self.dockfile =  './results/' + protentry + '_' + ligentry + '.out' # !!
        self.complex_name = protentry + '_' + ligentry + '_cplx.pdb'

    def dock_merge_plip(self):
        df = PandasPdb().read_pdb(self.tmpdir + '/' + self.protpdb)  # opens protein to calculate grid
        minx = df.df['ATOM']['x_coord'].min()
        maxx = df.df['ATOM']['x_coord'].max()
        cent_x = round((maxx + minx) / 2, 2)
        size_x = round(abs(maxx - minx) + 3, 2)
        miny = df.df['ATOM']['y_coord'].min()
        maxy = df.df['ATOM']['y_coord'].max()
        cent_y = round((maxy + miny) / 2, 2)
        size_y = round(abs(maxy - miny) + 3, 2)
        minz = df.df['ATOM']['z_coord'].min()
        maxz = df.df['ATOM']['z_coord'].max()
        cent_z = round((maxz + minz) / 2, 2)
        size_z = round(abs(maxz - minz) + 3, 2)
        assert (type(cent_x) != None), "Protein structure is damaged"
        assert (type(cent_y) != None), "Protein structure is damaged"
        assert (type(cent_z) != None), "Protein structure is damaged"


        print("Center point of docking grid for {} is as follows: "
              "x: {}, y: {}, z: {}".format(self.protein, size_x, size_y, size_z))
        print("Sizes of docking grid are as follows:"
              "x: {}, y: {}, z: {}".format(cent_x, cent_y, cent_z))
        print("Receptor: ", self.protein, "\n", "Ligand: ", self.ligand, "\n", "Output file: ", self.dockfile)
        os.system(
            'vina --receptor {} --ligand "{}" --center_x {} --center_y {} --center_z {} --size_x {} --size_y {} --size_z {} --log "{}" --out "{}"'.format(
                self.protein,
                self.ligand,
                cent_x,
                cent_y,
                cent_z,
                size_x,
                size_y,
                size_z,
                self.docklog,
                self.dockfile
            ))
        """Postprocessing of docking files"""
        # time.sleep(60)
        df.df['ATOM']['segment_id'].replace(r'.{1,}', '', regex=True, inplace=True) # Clean pdbqt inheritance
        df.df['ATOM']['blank_4'].replace(r'.{1,}', '', regex=True, inplace=True)
        docking_output_df = PandasPdb().read_pdb(self.dockfile)
        docking_output_df.df['HETATM'].drop_duplicates(subset='atom_number', keep='first', inplace=True)  # extract first model
        docking_output_df.df['HETATM']['segment_id'].replace(r'.{1,}', '', regex=True, inplace=True)
        docking_output_df.df['HETATM']['element_symbol'].replace('A', 'C', inplace=True) # OpenBabel fix
        docking_output_df.df['HETATM']['blank_4'].replace(r'.{1,}', '', regex=True, inplace=True)  # clean pdbqt inheritance
        df.df['ATOM'] = df.df['ATOM'].append(docking_output_df.df['HETATM'], ignore_index=True) # merges the files
        df.to_pdb(path = self.complex_name,
                  records=['ATOM','HETATM'],
                  gz = False,
                  append_newline=True)
        shutil.move(self.complex_name, './results')

In [33]:
from Bio import SwissProt
import tempfile 
#Dummy CID:
# top_cid=[6500, 6501]

top_cid=[str(x) for x in top_cid]
# check_id='POC6X7'
# Part 1 - look for PDB entry

list_of_pdbs = []
handle = ExPASy.get_sprot_raw(check_id)
record = SwissProt.read(handle)
for i in record.cross_references:
    if i[0] == 'PDB':
        list_of_pdbs.append(i[1].upper())
        
print("List of found pdbs: ", list_of_pdbs)
#Part 2 - download and prepare PDB
prepare_protein_path = '~/MGLTools-1.5.6/bin/pythonsh ~/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py -A bonds_hydrogens -U nphs_lps_waters_nonstdres_deleteAltB -r'
with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
    for entry in list_of_pdbs:
        try:
            ProteinPreparer(entry.lower(), tmpdir).prepare_protein()
            break
        except:
            pass
#Part 3 - prepare ligands
    for CID_entry in top_cid:
        try:
            LigandPreparer(CID_entry,tmpdir).prepare_ligand()
        except Exception as e:
            print(e)
#Part 4 - dock the ligand
        try:
            for entry in os.listdir(tmpdir):
                if (entry[0:4].upper() in list_of_pdbs) and (entry.endswith('.pdbqt')):
                    VinaDocker(protentry=entry[0:4].lower(),
                                        ligentry=CID_entry,
                                        protein_pdbqt=tmpdir + '/' + entry[0:4].lower(),
                                        ligand_pdbqt=tmpdir + '/' + CID_entry,
                                        dir=tmpdir
                                        ).dock_merge_plip()
        except Exception as e:
            print(e)
            
            
            
            
print("****Docking procedure finished****")

List of found pdbs:  ['1A8M', '1TNF', '2AZ5', '2E7A', '2TUN', '2ZJC', '2ZPX', '3ALQ', '3IT8', '3L9J', '3WD5', '4G3Y', '4TSV', '4TWT', '4Y6O', '5M2I', '5M2J', '5M2M', '5MU8', '5TSW', '5UUI', '5WUX', '5YOY', '6OOY', '6OOZ', '6OP0', '6RMJ']


/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt 
 Ligand:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/148332828.pdbqt 
 Output file:  ./results/1a8m_148332828.out
Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt 
 Ligand:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/24830655.pdbqt 
 Output file:  ./results/1a8m_24830655.out
Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Re

Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt 
 Ligand:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/18591132.pdbqt 
 Output file:  ./results/1a8m_18591132.out
Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt 
 Ligand:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/703093.pdbqt 
 Output file:  ./results/1a8m_703093.out
Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor

Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt 
 Ligand:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/5761940.pdbqt 
 Output file:  ./results/1a8m_5761940.out
Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt 
 Ligand:  /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/3116128.pdbqt 
 Output file:  ./results/1a8m_3116128.out
Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp12248oxo/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Receptor

In [ ]:
#Part 5 - PLIP
#protein-ligand interaction profiling through PLIP 
options=Options()
options.headless = True
plip = webdriver.Firefox(options=options)
plip.get("https://projects.biotec.tu-dresden.de/plip-web/plip")
print("Connection successful")

for cplx in os.listdir('./results/'):
    if cplx.endswith('cplx.pdb'):
        cplx=f'./results/{cplx}'
        print("Processing file {}".format(cplx))
    
        select_pdb_input = plip.find_element_by_xpath("//*[@id='select-pdb-by-file']").click() 

        browse = plip.find_element_by_xpath(
            '/html/body/div[1]/div[2]/div/form/div[1]/div[1]/div[3]/input'
        ).send_keys(                                                       
            os.getcwd()+'/{}'.format(cplx)
        )

        send_file = plip.find_element_by_xpath("//*[@id='submit']").click() 
        time.sleep(10) 
        try:
            try:
                open_interactions_1 = plip.find_element_by_xpath('/html/body/div/div[2]/div/div[1]/h2[2]').click()
                open_interactions_2 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[2]/h3').click()
                open_interactions_3 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[2]/div/h4').click()
                pngs = plip.find_elements_by_xpath("//a[contains(@href,'.png')]")
                pymolsessions = plip.find_elements_by_xpath("//a[contains(@href,'.pse')]")
                
            except:
                open_interactions_1 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/h2[1]').click()
                open_interactions_2 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[1]/h3').click()
                open_interactions_3 = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div[1]/div/h4').click()
                pngs = plip.find_elements_by_xpath("//a[contains(@href,'.png')]")
                pymolsessions = plip.find_elements_by_xpath("//a[contains(@href,'.pse')]")

            for image in pngs:
                print(image.get_attribute("href"))
                output_image = requests.get(image.get_attribute("href"))
                open(
                    os.getcwd()+'/{}'.format(cplx+'.png'), 'wb'
                ).write(output_image.content)
                print("Image saved as {}".format(cplx+'.png'))

            for pysession in pymolsessions:
                print(pysession.get_attribute("href"))
                pse = requests.get(pysession.get_attribute("href"))
                open(
                    os.getcwd()+'/{}'.format(cplx+'.pse'), 'wb'
                ).write(pse.content)
                print("Pymol sessions saved as {}".format(cplx+'.pse'))
                  
            restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/p[3]/a').click()
            time.sleep(5)
        except:
            print("No interactions found for {} or damaged structure".format(cplx))
            try:
                restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/p[3]/a').click()
                time.sleep(5)
            except:
                restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/p/a').click() 

Generative LSTM neural network based DeNovo drug lead compound generation

In [ ]:
import pandas as pd
import pubchempy as pc
from collections import OrderedDict
import os

with open("./datasets/dataset.smi", "w") as f:
        for cid in n_cid:
            f.write(pc.Compound.from_cid(cid).canonical_smiles + "\n")
            

In [195]:
a = len(n_cid)
b = a*0.001
c = int(b)
with open("./datasets/fine-tune.smi", "w") as f:
    i=0
    while i in range(c):
        f.write(pc.Compound.from_cid(n_cid[i]).canonical_smiles + "\n")
        i+=1

In [186]:
import os
os.system('python3 cleanup_smiles.py datasets/dataset.smi datasets/dataset_cleansed.smi')

0

In [196]:
import os
os.system('python3 cleanup_smiles.py datasets/fine-tune.smi datasets/cleansed_fine-tune.smi')

0

In [6]:
import os
os.system('python train.py')

0

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from lstm_chem.utils.config import process_config
from lstm_chem.model import LSTMChem
from lstm_chem.generator import LSTMChemGenerator

In [2]:
CONFIG_FILE = 'experiments/2020-12-10/LSTM_Chem/config.json'
config = process_config(CONFIG_FILE)

In [3]:
modeler = LSTMChem(config, session='generate')

Loading model architecture from experiments/2020-12-10/LSTM_Chem/model_arch.json ...
Loading model checkpoint from experiments/2020-12-10/LSTM_Chem/checkpoints/LSTM_Chem-30-0.36.hdf5 ...
Loaded the Model.


In [4]:
# It took more than 4 hours!
generator = LSTMChemGenerator(modeler)
sampled_smiles = generator.sample(num=500)

100%|██████████| 500/500 [02:52<00:00,  2.90it/s]


In [5]:
Generated_Smiles = []

In [6]:
from rdkit import RDLogger, Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
RDLogger.DisableLog('rdApp.*')


In [43]:
valid_mols = []
for smi in sampled_smiles:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        valid_mols.append(mol)
print('Number of Valid Smiles out of the 500 generated by the neural network is')
print(len(valid_mols))

Number of Valid Smiles out of the 500 generated by the neural network is
182


In [11]:
i=0
while i in range(len(valid_mols)):
    Generated_Smiles.append(Chem.MolToSmiles(valid_mols[i],isomericSmiles=False))
    i+=1

In [42]:
A = Generated_Smiles[:len(Generated_Smiles)//2]
B = Generated_Smiles[len(Generated_Smiles)//2:]
A1 = A[:len(A)//2]
A2 = A[len(A)//2:]
B1 = B[:len(B)//2]
B2 = B[len(B)//2:]

Print('The Generated SMILES of compounds with drug lead potential')
Generated_Smiles

['O=C(Cc1ccc(Cl)cn1)c1ccc(Cl)c(S(=O)(=O)N2CCOCC2)c1',
 'COC(=O)c1ccccc1C(=O)N1CCN(C2CCc3ccccc3S2)CC1',
 'CCC1CCCCC1C(=O)CSC1=NC2=CC=C(OCc3ccccc3)C2=C1',
 'Cc1ccccc1NC(=S)NNc1ncc(C(F)(F)F)cc1Cl',
 'CCOC(=O)c1ccccc1C(=O)c1ccccc1C(F)(F)F',
 'COc1ccc(CC(=O)c2cc(C)n(Cc3ccccc3)c2C)cc1',
 'COc1ccc(C(=O)c2cc(C)n(Cc3cccs3)c2)cc1',
 'COc1cc(-c2cc3ccccc3sc2=O)ccc1NC(=O)c1ccccc1OC',
 'CC(C)(C)c1nn(CCC(=O)Nc2ccccc2)c2ccccc12',
 'Cc1ccc(C(=O)CNc2ncc(-c3ccccc3)s2)cc1',
 'CC(=O)Nc1ccc(C(=O)Nc2ccc(C(F)(F)F)cc2)cc1',
 'Cn1c(=O)c2c(nc(N3CCCC(O)C3)n2Cc2ccccc2)n(C)c1=O',
 'Cc1ccc(S(=O)(=O)Nc2ccc(C(=O)Nc3ccccc3C)cc2)cc1',
 'C=CCN1Sc2cc(C(C)C)nnc2NC(=O)c2ccccc2S1(=O)=O',
 'CCCCCCC(=O)Nc1cc(-c2ccc(Cl)cc2)cs1',
 'COc1ccc(NC(=O)c2ccccc2Oc2ccc(C)cc2)cc1',
 'CC(C)(C)Cc1ccc(-c2csc(NC(=O)c3ccc(C(=O)O)cc3)n2)cc1',
 'CC(=O)c1ccc(NC(=O)c2cccc([N+](=O)[O-])c2)cc1',
 'CCN(c1ccccc1)S(=O)(=O)c1ccc(C(=O)Nc2ccc3c(c2)OCCO3)cc1',
 'COc1ccc(-c2nnc(N(C)C)s2)cc1',
 'Cc1ccccc1NC(=O)c1cccc(CS(=O)(=O)c2ccccc2)c1',
 'CN(C)CC(=O)Nc1c

Automated In Silico modelling of interaction of the drug target and compounds not registered on PubChem, generated as drug leads by the generative LSTM model

In [ ]:
"""Repeated & modified docking procedure for generated, custom smiles"""
#set preparation pathways
prepare_protein_path = '~/MGLTools-1.5.6/bin/pythonsh ~/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py -A bonds_hydrogens -U nphs_lps_waters_nonstdres_deleteAltB -r'

"""Class inheriting from Select checking if the residue is aminoacid"""
class ResSelect(Select):
    def accept_residue(self, residue):
        if is_aa(residue):
            return 1
        else:
            return 0


class ProteinPreparer:

    def __init__(self, protein: str, dir: str):
        self.protein = protein
        self.tmpdir = dir
        self.filename = self.tmpdir + '/pdb{}.ent'.format(self.protein)

    def prepare_protein(self):
        parser = PDBParser()
        io = PDBIO()
        PDBList().retrieve_pdb_file(pdb_code=self.protein, pdir=self.tmpdir,
                                    file_format='pdb')  # saves pdb in a form of ent file
        ipdb = parser.get_structure('ipdb', self.filename)  # Input pdb as a self.filename
        io.set_structure(ipdb)  # Setting structure for input pdb
        io.save(self.tmpdir + '/'+self.protein + '.pdb', ResSelect(),
                preserve_atom_numbering=True)  # saves the cleaned pdb
        os.system(prepare_protein_path + self.tmpdir + '/{}.pdb'.format(
            self.protein))  # prepares the structure by adding polar hydrogens and adding gesteiger charges
        shutil.move(self.protein + '.pdbqt', self.tmpdir)

class ObtainInteractionsFromComplex:
    def __init__(self, protein_ligand_complex):
        self.cplx = protein_ligand_complex

    def connect_retrieve(self):
        try:
            options = Options()
            options.headless = True
            plip = webdriver.Firefox(options=options)
            plip.get("https://projects.biotec.tu-dresden.de/plip-web/plip")
            print("Connection to PLIP server successful")

            print("Processing file {}".format(self.cplx))
            select_pdb_input = plip.find_element_by_xpath(
                "//*[@id='select-pdb-by-file']").click()  # Click upload pdb

            browse = plip.find_element_by_xpath(
                '/html/body/div[1]/div[2]/div/form/div[1]/div[1]/div[3]/input'
            ).send_keys(  # search for browse button and choose complex
                os.getcwd() + '/{}'.format(self.cplx)
            )

            send_file = plip.find_element_by_xpath("//*[@id='submit']").click()  # apply operation
            time.sleep(10)  # wait for some time by the time connection is stabilized
            try:
                try:
                    open_interactions_1 = plip.find_element_by_xpath(
                        '/html/body/div/div[2]/div/div[1]/h2[2]').click()
                    open_interactions_2 = plip.find_element_by_xpath(
                        '/html/body/div[1]/div[2]/div/div[1]/div[2]/h3').click()
                    open_interactions_3 = plip.find_element_by_xpath(
                        '/html/body/div[1]/div[2]/div/div[1]/div[2]/div/h4').click()
                    pngs = plip.find_elements_by_xpath("//a[contains(@href,'.png')]")
                    pymolsessions = plip.find_elements_by_xpath("//a[contains(@href,'.pse')]")

                except:
                    open_interactions_1 = plip.find_element_by_xpath(
                        '/html/body/div[1]/div[2]/div/div[1]/h2[1]').click()
                    open_interactions_2 = plip.find_element_by_xpath(
                        '/html/body/div[1]/div[2]/div/div[1]/div[1]/h3').click()
                    open_interactions_3 = plip.find_element_by_xpath(
                        '/html/body/div[1]/div[2]/div/div[1]/div[1]/div/h4').click()
                    pngs = plip.find_elements_by_xpath("//a[contains(@href,'.png')]")
                    pymolsessions = plip.find_elements_by_xpath("//a[contains(@href,'.pse')]")

                for image in pngs:
                    print(image.get_attribute("href"))
                    output_image = requests.get(image.get_attribute("href"))
                    open(
                        os.getcwd() + '/results/{}'.format(self.cplx + '.png'), 'wb'
                    ).write(output_image.content)
                    print("Image saved as {}".format(self.cplx + '.png'))

                for pysession in pymolsessions:
                    print(pysession.get_attribute("href"))
                    pse = requests.get(pysession.get_attribute("href"))
                    open(
                        os.getcwd() + '/results/{}'.format(self.cplx + '.pse'), 'wb'
                    ).write(pse.content)
                    print("Pymol sessions saved as {}".format(self.cplx + '.pse'))

                restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/p[3]/a').click()
                time.sleep(5)

            except:
                print("No interactions found for {} or damaged structure".format(self.cplx))
                try:
                    restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div/p[3]/a').click()
                    time.sleep(5)
                except:
                    restart_plip = plip.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/p/a').click()
        except Exception as e:
            print(e)
            pass

class SmileVinaDocker:

    def __init__(self, ligentry: str, protentry: str, protein_pdbqt: str, ligand_pdbqt: str, dir: str):
        self.protein = protein_pdbqt + '.pdbqt'
        self.protpdb = protentry + '.pdb'
        self.protname = os.path.basename(self.protein)
        self.ligand = ligand_pdbqt + '.pdbqt'
        self.ligname = os.path.basename(self.ligand)
        self.tmpdir = dir
        self.docklog = './results/' + protentry + '_' + ligentry + '_docking.log'
        self.dockfile =  './results/' + protentry + '_' + ligentry + '.out'
        self.complex_name = protentry + '_' + ligentry + '_cplx.pdb'

    def dock_merge_plip(self):
        df = PandasPdb().read_pdb(self.tmpdir + '/' + self.protpdb)  # opens protein to calculate grid
        minx = df.df['ATOM']['x_coord'].min()
        maxx = df.df['ATOM']['x_coord'].max()
        cent_x = round((maxx + minx) / 2, 2)
        size_x = round(abs(maxx - minx) + 3, 2)
        miny = df.df['ATOM']['y_coord'].min()
        maxy = df.df['ATOM']['y_coord'].max()
        cent_y = round((maxy + miny) / 2, 2)
        size_y = round(abs(maxy - miny) + 3, 2)
        minz = df.df['ATOM']['z_coord'].min()
        maxz = df.df['ATOM']['z_coord'].max()
        cent_z = round((maxz + minz) / 2, 2)
        size_z = round(abs(maxz - minz) + 3, 2)
        assert (type(cent_x) != None), "Protein structure is damaged"
        assert (type(cent_y) != None), "Protein structure is damaged"
        assert (type(cent_z) != None), "Protein structure is damaged"


        print("Center point of docking grid for {} is as follows: "
              "x: {}, y: {}, z: {}".format(self.protein, size_x, size_y, size_z))
        print("Sizes of docking grid are as follows:"
              "x: {}, y: {}, z: {}".format(cent_x, cent_y, cent_z))
        os.system(
            'vina --receptor {} --ligand "{}" --center_x {} --center_y {} --center_z {} --size_x {} --size_y {} --size_z {} --log "{}" --out "{}"'.format(
                self.protein,
                self.ligand,
                cent_x,
                cent_y,
                cent_z,
                size_x,
                size_y,
                size_z,
                self.docklog,
                self.dockfile
            ))
        """Postprocessing of docking files"""

        df.df['ATOM']['segment_id'].replace(r'.{1,}', '', regex=True, inplace=True) # Clean pdbqt inheritance
        df.df['ATOM']['blank_4'].replace(r'.{1,}', '', regex=True, inplace=True)
        docking_output_df = PandasPdb().read_pdb(self.dockfile)
        docking_output_df.df['HETATM'].drop_duplicates(subset='atom_number', keep='first', inplace=True)  # extract first model
        docking_output_df.df['HETATM']['segment_id'].replace(r'.{1,}', '', regex=True, inplace=True)
        docking_output_df.df['HETATM']['element_symbol'].replace('A', 'C', inplace=True) # OpenBabel fix
        docking_output_df.df['HETATM']['blank_4'].replace(r'.{1,}', '', regex=True, inplace=True)  # clean pdbqt inheritance
        df.df['ATOM'] = df.df['ATOM'].append(docking_output_df.df['HETATM'], ignore_index=True) # merges the files
        df.to_pdb(path = self.complex_name,
                  records=['ATOM','HETATM'],
                  gz = False,
                  append_newline=True)
        try:
            ObtainInteractionsFromComplex(self.complex_name).connect_retrieve() # PLIP analysis
        except:
            pass
        shutil.move(self.complex_name, './results')

#set preparation pathways
prepare_ligand_path = '~/MGLTools-1.5.6/bin/pythonsh ~/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py -A bonds_hydrogens -U nphs_lps -l'

class SmileLigandPreparer:

    def __init__(self, smile: str, dir: str):
        self.ligand = smile
        self.tmpdir = dir

    def prepare_ligand(self):
        with open(self.tmpdir+'/'+self.ligand+'.smi', 'w') as smile_input_file:
            smile_input_file.write(self.ligand)

        os.system('obabel "{}" -O "{}" --gen3d'.format(
            self.tmpdir + '/' + self.ligand + '.smi', self.tmpdir + '/' + self.ligand + 'prep.pdb'))
        # generates 3d coords
        os.system(
            'obminimize -ff GAFF "{}" > "{}"'.format(
                self.tmpdir + '/' + self.ligand + 'prep.pdb', self.tmpdir + '/' + self.ligand + '.pdb'
                                                 )
        )
        #minimizes using GAFF
        os.system(prepare_ligand_path + '"{}"'.format(self.tmpdir + '/' + self.ligand + '.pdb'
                                                    )) # adds charges, sets rotatable bonds
        shutil.move(self.ligand + '.pdbqt', self.tmpdir)
class PLIP_modifier:
    def __init__(self, file):
        self.file = file

    def customize_plip_pse(self) -> str:
        try:
            print(f"Opening the file {self.file}")
            p.cmd.load(self.file)
            p.cmd.show(representation="sticks", selection=" resn UNL")
            p.cmd.show(representation="spheres", selection=" resn UNL")
            p.cmd.color(color='gray85', selection="resn UNL and elem C")
            p.cmd.color(color='red', selection="resn UNL and elem O")
            p.cmd.color(color='slate', selection="resn UNL and elem N")
            p.cmd.color(color='gray85', selection="resn UNL and elem H")
            p.cmd.set('stick_color','black','resn UNL')
            p.cmd.set('stick_radius','.01','resn UNL')
            p.cmd.set( 'ray_texture', '2')
            p.cmd.set( 'antialias', '3')
            p.cmd.set( 'sphere_scale', '.25')
            p.cmd.set( 'ambient', '0.5')
            p.cmd.set( 'sphere_scale', '.13', 'elem H ')
            p.cmd.set( 'spec_count', '5')
            p.cmd.set( 'bg_rgb','[1, 1, 1]')
            p.cmd.set( 'shininess', '50')
            p.cmd.set( 'stick_quality', '50')
            p.cmd.set( 'specular', '1')
            p.cmd.set( 'sphere_quality', '4')
            p.cmd.set( 'reflect',' .1')
            p.cmd.set( 'ray_trace_mode', '1')
            p.cmd.set( 'dash_gap', '0.2')
            p.cmd.set( 'dash_color', 'blue')
            p.cmd.set( 'dash_gap', '.3')
            p.cmd.set( 'dash_length', '.05')
            p.cmd.set( 'dash_round_ends', '0')
            p.cmd.color("grey", "resn UNL and e. C")
            p.cmd.label2("visible and polymer and n. ca", "%s:%s" % ('resn','resi'))
            p.cmd.set( "label_color", "black")
            p.cmd.set( "label_position","(0,0,5)")
            p.cmd.set( "label_font_id", "7")
            p.cmd.set( "label_size", "-0.7")
            p.cmd.bg_color("white")
            p.cmd.set('ray_opaque_background', '1')
            p.cmd.save(f"./results/{self.file}_customized.png")
            p.cmd.save(f"./results/{self.file}_customized.pse")
            print(f"File {self.file} successfully converted and saved as _customized")
        except Exception as e:
            print(e)
            pass




In [36]:
PDB = []
PDB.append(list_of_pdbs[0])
PDB



['1A8M']

In [1]:
home = os.path.expanduser("~")

def execute(PDB_list: list, CID_list: list):
    assert (os.path.isdir(
        home + '/MGLTools-1.5.6')), "AutoDockTools not found! Script stopped"  # check if mgltools are present

    for CID_entry in CID_list:
        with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
            SmileLigandPreparer(CID_entry, dir=tmpdir).prepare_ligand()
            for PDB_entry in PDB_list:
                ProteinPreparer(PDB_entry.lower(), dir=tmpdir).prepare_protein()
                SmileVinaDocker(protentry=PDB_entry.lower(),
                                           ligentry=CID_entry,
                                           protein_pdbqt=tmpdir + '/' + PDB_entry.lower(),
                                           ligand_pdbqt=tmpdir + '/' + CID_entry,
                                           dir=tmpdir
                                           ).dock_merge_plip()

                    

In [ ]:
for file in os.listdir('./results/'):
    if file.endswith('cplx.pdb.pse'):
        PLIP_modifier(file).customize_plip_pse()



In [38]:
execute(PDB, A1)



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpz9tzsxwl/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_O=C(Cc1ccc(Cl)cn1)c1ccc(Cl)c(S(=O)(=O)N2CCOCC2)c1_cplx.pdb
No interactions found for 1a8m_O=C(Cc1ccc(Cl)cn1)c1ccc(Cl)c(S(=O)(=O)N2CCOCC2)c1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp06gio67_/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_COC(=O)c1ccccc1C(=O)N1CCN(C2CCc3ccccc3S2)CC1_cplx.pdb
No interactions found for 1a8m_COC(=O)c1ccccc1C(=O)N1CCN(C2CCc3ccccc3S2)CC1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


...

Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpdijwbyly/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_O=C(Nc1nc(-c2ccc(Cl)cc2)cs1)c1ccc(Br)c(S(=O)(=O)N2CCOCC2)c1_cplx.pdb
No interactions found for 1a8m_O=C(Nc1nc(-c2ccc(Cl)cc2)cs1)c1ccc(Br)c(S(=O)(=O)N2CCOCC2)c1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp_eoer5dh/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_Cc1ccc(S(=O)(=O)NCc2cccnc2)cc1_cplx.pdb
No interactions found for 1a8m_Cc1ccc(S(=O)(=O)NCc2cccnc2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpvi0xdqs6/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_O=C(NCCC(=S)N1CCN(c2ccccc2)CC1)c1ccccc1_cplx.pdb
No interactions found for 1a8m_O=C(NCCC(=S)N1CCN(c2ccccc2)CC1)c1ccccc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



In [39]:
execute(PDB, A2)

/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp9mntemx5/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_CC(C)CCn1c(N)c(C(=O)COC(=O)C23CC4CC(CC(C4)C2)C3)c(=O)n(C)c1=O_cplx.pdb
No interactions found for 1a8m_CC(C)CCn1c(N)c(C(=O)COC(=O)C23CC4CC(CC(C4)C2)C3)c(=O)n(C)c1=O_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp_ja2guur/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_CC(C)c1ccc(S(=O)(=O)N2CCC(c3ccccc3)CC2)cc1_cplx.pdb
No interactions found for 1a8m_CC(C)c1ccc(S(=O)(=O)N2CCC(c3ccccc3)CC2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


...

Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpcq26spgk/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_CCN(CC)C(=O)C(C#N)=Cc1ccc(C(=O)OCc2ccccc2)cc1_cplx.pdb
No interactions found for 1a8m_CCN(CC)C(=O)C(C#N)=Cc1ccc(C(=O)OCc2ccccc2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmps9vwkrkg/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_Cc1ccc(NC(=O)C=C(S)c2ccccc2C(=O)c2ccccc2)cc1_cplx.pdb
No interactions found for 1a8m_Cc1ccc(NC(=O)C=C(S)c2ccccc2C(=O)c2ccccc2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpxp2lcgkn/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_COc1ccc(NC(=O)c2cccc([N+](=O)[O-])c2)cc1_cplx.pdb
No interactions found for 1a8m_COc1ccc(NC(=O)c2cccc([N+](=O)[O-])c2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



In [40]:
execute(PDB, B1)

/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpmpxn1f6r/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_O=C(CCc1cccc(Cl)c1)Nc1nc(-c2ccccc2)cs1_cplx.pdb
No interactions found for 1a8m_O=C(CCc1cccc(Cl)c1)Nc1nc(-c2ccccc2)cs1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpdl6covbz/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_COc1ccccc1C(=O)Nc1ccc(-c2cccc([N+](=O)[O-])c2)cc1_cplx.pdb
No interactions found for 1a8m_COc1ccccc1C(=O)Nc1ccc(-c2cccc([N+](=O)[O-])c2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


...

Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpd3hr7zhc/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_Cc1ccc(S(=O)(=O)N2CCCC2)cc1OCC(=O)Nc1ccc(Cl)c(Cl)c1_cplx.pdb
No interactions found for 1a8m_Cc1ccc(S(=O)(=O)N2CCCC2)cc1OCC(=O)Nc1ccc(Cl)c(Cl)c1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp36s6j4tk/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_Cc1ccc(S(=O)(=O)Nc2cccc(C)c2)cc1_cplx.pdb
No interactions found for 1a8m_Cc1ccc(S(=O)(=O)Nc2cccc(C)c2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpkrbxmki_/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_CC1CC(C(=O)c2ccc(Cl)c(Cl)c2)c2ccccc21_cplx.pdb
No interactions found for 1a8m_CC1CC(C(=O)c2ccc(Cl)c(Cl)c2)c2ccccc21_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



In [41]:
execute(PDB, B2)

/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpzzylp6vf/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_COc1ccc(NC(=O)c2ccccc2C(=O)c2ccccc2)cc1_cplx.pdb
No interactions found for 1a8m_COc1ccc(NC(=O)c2ccccc2C(=O)c2ccccc2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmppjlqlm8_/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_Cc1ccc(C(=O)NC2CCN(C(=O)c3ccccc3)C2)cc1_cplx.pdb
No interactions found for 1a8m_Cc1ccc(C(=O)NC2CCN(C(=O)c3ccccc3)C2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


...

Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpaedx_u0d/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_O=C(Nc1ccc(S(=O)(=O)N2CCOCC2)cc1)c1cccc(Cl)c1_cplx.pdb
No interactions found for 1a8m_O=C(Nc1ccc(S(=O)(=O)N2CCOCC2)cc1)c1cccc(Cl)c1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmp7esba57z/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_CCCc1nc2ccc(NC(=O)c3ccc(F)cc3)ccnc1-2_cplx.pdb
No interactions found for 1a8m_CCCc1nc2ccc(NC(=O)c3ccc(F)cc3)ccnc1-2_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4109.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4125.
  PDBConstructionWarning,
/home/ben/anaconda3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4150.
  PDBConstructionWarning,


Center point of docking grid for /home/ben/Documents/Target2DeNovoDrug/tmpdm5av67r/1a8m.pdbqt is as follows: x: 64.23, y: 65.34, z: 63.32
Sizes of docking grid are as follows:x: 20.25, y: 50.86, z: 41.17
Connection to PLIP server successful
Processing file 1a8m_COC(=O)c1ccc(S(=O)(=O)N2CCN(c3ncccn3)CC2)cc1_cplx.pdb
No interactions found for 1a8m_COC(=O)c1ccc(S(=O)(=O)N2CCN(c3ncccn3)CC2)cc1_cplx.pdb or damaged structure
Message: Unable to locate element: /html/body/div[1]/div[2]/div[2]/p/a



In [ ]:
import os
os.chdir('/home/ben/Documents/TargetDeNovoDrug with MD/results')
os.system('bash runGromacs.sh')

Project ends